In [1]:
import pymysql
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout

Using TensorFlow backend.


In [2]:
_host = '168.131.39.95'
_port = 3306
_user = 'root'
_passwd = 'jslab0963'
_db = 'TYP'
db = pymysql.connect(host=_host, port=_port, user=_user, passwd=_passwd, db=_db, charset= 'utf8')
cursor = db.cursor()
typ_name = ['CHABA2', 'CHANHOM', 'GONI', 'HALOLA', 'HALONG', 'KONGREY2', 'MALAKAS', 'NAKRI2', 'NANGKA', 'NANMADOL', 'NEOGURI',
           'NORU', 'PRAPIROON', 'RUMBIA', 'SOULIK', 'TALIM', 'TRAMI', 'VONGFONG']

In [3]:
#데이터 전처리
typ_feat = []
typ_lang = []

for name in typ_name:
    sql = " SELECT * from " + name
    cursor.execute(sql)
    result = cursor.fetchall()
    for row_data in result:
        typ_lang.append([row_data[1], row_data[2]])
        typ_feat.append([row_data[3], row_data[4], row_data[5]])

#typ_feat : 수온, 풍향, 풍속의 feature들 x_data
print(typ_feat)

#typ_lang : 대한민국 역대 태풍들의 위, 경도 y_data
print(typ_lang)

[[110, Decimal('26.000'), Decimal('29.000')], [115, Decimal('29.000'), Decimal('29.400')], [135, Decimal('29.000'), Decimal('29.400')], [145, Decimal('23.000'), Decimal('29.500')], [145, Decimal('26.000'), Decimal('29.500')], [150, Decimal('27.000'), Decimal('29.500')], [160, Decimal('27.000'), Decimal('29.500')], [175, Decimal('26.000'), Decimal('29.100')], [145, Decimal('30.000'), Decimal('29.500')], [190, Decimal('25.000'), Decimal('28.700')], [165, Decimal('25.000'), Decimal('28.600')], [165, Decimal('27.000'), Decimal('29.000')], [170, Decimal('35.000'), Decimal('29.400')], [170, Decimal('44.000'), Decimal('28.700')], [190, Decimal('56.000'), Decimal('28.900')], [215, Decimal('61.000'), Decimal('28.600')], [210, Decimal('56.000'), Decimal('28.900')], [215, Decimal('57.000'), Decimal('29.200')], [205, Decimal('55.000'), Decimal('29.000')], [205, Decimal('52.000'), Decimal('29.200')], [185, Decimal('50.000'), Decimal('29.100')], [205, Decimal('56.000'), Decimal('29.100')], [200, Dec

In [4]:
#MinMaxScaler으로 feature 값을 0부터 1까지 변환
typ_feat = np.asarray(typ_feat, dtype = np.float32)
scaler = MinMaxScaler(feature_range=(0., 1.))
typ_feat = scaler.fit_transform(typ_feat)
print(len(typ_feat))
print(typ_feat)

typ_lang = np.asarray(typ_lang, dtype = np.float32)
print(len(typ_lang))
print(typ_lang)

#train, value, test 값으로 나누기
x_train = typ_feat[0:260]
x_val = typ_feat[260:520]
x_test = typ_feat[520:784]

y_train = typ_lang[0:260]
y_val = typ_lang[260:520]
y_test = typ_lang[520:784]

784
[[0.2857143  0.13872832 0.84939766]
 [0.3        0.15606935 0.8734939 ]
 [0.35714287 0.15606935 0.8734939 ]
 ...
 [0.87142855 0.11560693 0.        ]
 [0.85714287 0.15606935 0.01807231]
 [0.9        0.29479766 0.03614461]]
784
[[ 15.8 158.1]
 [ 15.6 156.4]
 [ 15.2 154.6]
 ...
 [ 34.5 135.6]
 [ 36.7 139.5]
 [ 39.6 143.2]]


In [5]:
db.close()

In [6]:
print(len(typ_lang))
print(len(typ_feat))

784
784


In [7]:
# 데이터셋 전처리
# list 형태 바꾸기
x_train = np.reshape(x_train, (-1, 3, 1))
x_val = np.reshape(x_val, (-1, 3, 1))
x_test = np.reshape(x_test, (-1, 3, 1))

In [8]:
y_train = np.reshape(y_train, (-1, 2))
y_val = np.reshape(y_train, (-1, 2))
y_test = np.reshape(y_test, (-1, 2))

In [ ]:
print(x_train.shape)
print(y_train.shape)

(260, 3, 1)
(260, 2)


In [ ]:
# 모델 선언
data_dim = 1
timesteps = 3

# stacked lstm
model = Sequential()
model.add(LSTM(260, input_shape=(3, 1), return_sequences = True, activation = 'relu'))
model.add(LSTM(260, activation = 'relu'))
model.add(Dense(2,activation='relu'))
print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 3, 260)            272480    
_________________________________________________________________
lstm_2 (LSTM)                (None, 260)               541840    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 522       
Total params: 814,842
Trainable params: 814,842
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# 모델 학습과정 설정하기
model.compile(loss='mean_squared_error', optimizer='adam')

# 모델 학습시키기
hist = model.fit(x_train, y_train, epochs=500, batch_size=10, validation_data=(x_val, y_val))
#hist = model.fit(x_train,y_train,epochs=300,batch_size=10,validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 260 samples, validate on 260 samples
Epoch 1/500
260/260 [==============================] - 5s 17ms/step - loss: 9857.2893 - val_loss: 9546.1177
Epoch 2/500
260/260 [==============================] - 0s 2ms/step - loss: 5410.6600 - val_loss: 1343.8658
Epoch 3/500
260/260 [==============================] - 1s 2ms/step - loss: 656.6276 - val_loss: 432.5028
Epoch 4/500
260/260 [==============================] - 1s 2ms/step - loss: 469.4943 - val_loss: 332.5155
Epoch 5/500
260/260 [==============================] - 0s 2ms/step - loss: 413.8290 - val_loss: 309.6635
Epoch 6/500
260/260 [==============================] - 0s 2ms/step - loss: 351.0146 - val_loss: 240.5508
Epoch 7/500
260/260 [==============================] - 1s 2ms/step - loss: 315.2506 - val_loss: 203.6882
Epoch 8/500
260/260 [==============================] - 0s 2ms/step - loss: 259.7090 - val_loss: 175.0344
Epoch 9/500
260/260 [==============================] - 0s 2ms

260/260 [==============================] - 0s 2ms/step - loss: 75.3550 - val_loss: 177.6549
Epoch 76/500
260/260 [==============================] - 0s 2ms/step - loss: 70.8619 - val_loss: 175.1570
Epoch 77/500
260/260 [==============================] - 0s 2ms/step - loss: 82.4302 - val_loss: 175.2251
Epoch 78/500
260/260 [==============================] - 0s 2ms/step - loss: 87.2493 - val_loss: 180.6592
Epoch 79/500
260/260 [==============================] - 0s 2ms/step - loss: 82.4089 - val_loss: 194.8373
Epoch 80/500
260/260 [==============================] - 0s 2ms/step - loss: 72.7301 - val_loss: 177.4345
Epoch 81/500
260/260 [==============================] - 0s 2ms/step - loss: 78.2181 - val_loss: 177.6571
Epoch 82/500
260/260 [==============================] - 0s 2ms/step - loss: 73.6839 - val_loss: 178.7989
Epoch 83/500
260/260 [==============================] - 0s 2ms/step - loss: 72.1509 - val_loss: 170.7187
Epoch 84/500
260/260 [==============================] - 0s 2ms/step 

260/260 [==============================] - 0s 2ms/step - loss: 60.5868 - val_loss: 178.6317
Epoch 153/500
260/260 [==============================] - 0s 2ms/step - loss: 59.2869 - val_loss: 184.1469
Epoch 154/500
260/260 [==============================] - 0s 2ms/step - loss: 62.3246 - val_loss: 182.3994
Epoch 155/500
260/260 [==============================] - 0s 2ms/step - loss: 60.0312 - val_loss: 181.5507
Epoch 156/500
260/260 [==============================] - 1s 2ms/step - loss: 67.2637 - val_loss: 184.0354
Epoch 157/500
260/260 [==============================] - 0s 2ms/step - loss: 63.5829 - val_loss: 198.6313
Epoch 158/500
260/260 [==============================] - 0s 2ms/step - loss: 58.3493 - val_loss: 178.0113
Epoch 159/500
260/260 [==============================] - 0s 2ms/step - loss: 58.8835 - val_loss: 183.8097
Epoch 160/500
260/260 [==============================] - 0s 2ms/step - loss: 58.0710 - val_loss: 179.7543
Epoch 161/500
260/260 [==============================] - 0s 

260/260 [==============================] - 1s 2ms/step - loss: 55.0141 - val_loss: 189.0797
Epoch 230/500
260/260 [==============================] - 0s 2ms/step - loss: 57.8955 - val_loss: 189.9237
Epoch 231/500
260/260 [==============================] - 0s 2ms/step - loss: 54.9553 - val_loss: 185.5392
Epoch 232/500
260/260 [==============================] - 0s 2ms/step - loss: 61.6000 - val_loss: 187.9374
Epoch 233/500
260/260 [==============================] - 0s 2ms/step - loss: 59.2015 - val_loss: 193.2005
Epoch 234/500
260/260 [==============================] - 0s 2ms/step - loss: 52.4625 - val_loss: 189.4500
Epoch 235/500
260/260 [==============================] - 0s 2ms/step - loss: 54.1004 - val_loss: 189.7324
Epoch 236/500
260/260 [==============================] - 0s 2ms/step - loss: 53.3114 - val_loss: 190.6112
Epoch 237/500
260/260 [==============================] - 0s 2ms/step - loss: 58.3381 - val_loss: 187.2669
Epoch 238/500
260/260 [==============================] - 0s 

260/260 [==============================] - 0s 2ms/step - loss: 52.1464 - val_loss: 187.7580
Epoch 307/500
260/260 [==============================] - 0s 2ms/step - loss: 53.5146 - val_loss: 189.4840
Epoch 308/500
260/260 [==============================] - 0s 2ms/step - loss: 55.3192 - val_loss: 189.6274
Epoch 309/500
260/260 [==============================] - 0s 2ms/step - loss: 54.2586 - val_loss: 199.5082
Epoch 310/500
260/260 [==============================] - 0s 2ms/step - loss: 58.6752 - val_loss: 189.8097
Epoch 311/500
260/260 [==============================] - 0s 2ms/step - loss: 54.2689 - val_loss: 186.7207
Epoch 312/500
260/260 [==============================] - 0s 2ms/step - loss: 52.4283 - val_loss: 188.1656
Epoch 313/500
260/260 [==============================] - 0s 2ms/step - loss: 50.9715 - val_loss: 186.2015
Epoch 314/500
260/260 [==============================] - 0s 2ms/step - loss: 51.4905 - val_loss: 190.9714
Epoch 315/500
260/260 [==============================] - 0s 

260/260 [==============================] - 0s 2ms/step - loss: 53.5743 - val_loss: 199.5265
Epoch 384/500
260/260 [==============================] - 0s 2ms/step - loss: 53.9814 - val_loss: 191.8957
Epoch 385/500
260/260 [==============================] - 0s 2ms/step - loss: 49.6250 - val_loss: 192.8843
Epoch 386/500
260/260 [==============================] - 0s 2ms/step - loss: 50.9861 - val_loss: 195.9868
Epoch 387/500
260/260 [==============================] - 0s 2ms/step - loss: 53.1477 - val_loss: 187.8412
Epoch 388/500
260/260 [==============================] - 0s 2ms/step - loss: 55.1234 - val_loss: 213.7649
Epoch 389/500
260/260 [==============================] - 0s 2ms/step - loss: 54.4326 - val_loss: 189.8216
Epoch 390/500
260/260 [==============================] - 0s 2ms/step - loss: 60.8124 - val_loss: 183.1895
Epoch 391/500
260/260 [==============================] - 0s 2ms/step - loss: 53.8270 - val_loss: 189.6255
Epoch 392/500
260/260 [==============================] - 0s 

In [ ]:
#학습 과정 살펴보기
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.ylim(0.0, 200.0)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
#모델 평가
trainScore = model.evaluate(x_train, y_train, verbose=0)
model.reset_states()
print('Train Score: ', trainScore)
valScore = model.evaluate(x_val, y_val, verbose=0)
model.reset_states()
print('Validataion Score: ', valScore)
testScore = model.evaluate(x_test, y_test, verbose=0)
model.reset_states()
print('Test Score: ', testScore)

In [ ]:
# chaba 태풍을 예시로 모델 평가
chaba_xdata = np.array([[110, 26.000, 29.000], [115,29.000,29.400], [135,29.000,29.400], [145,23.000,29.500], 
                  [145, 26.000, 29.500], [150, 27.000, 29.500], [160, 27.000, 29.500], [175, 26.000, 29.100],
                  [145, 30.000, 29.500], [190, 25.000, 28.700], [165, 25.000, 28.600], [165, 27.000, 29.000],
                  [170, 35.000, 29.400], [170, 44.000, 28.700], [190, 56.000, 28.900], [215, 61.000, 28.600],
                  [210, 56.000, 28.900], [215, 57.000, 29.200], [205, 55.000, 29.000], [205, 52.000, 29.200],
                  [185, 50.000, 29.100], [205, 56.000, 29.100], [200, 47.000, 28.700], [195, 49.000, 29.000], [195, 61.000, 29.000]]) #입력
chaba_realdata = np.array([[15.8, 158.1],[15.6, 156.4], [15.2, 154.6], [14.7, 153.1], [14.3, 152.0], [14.4, 151.4],
                  [14.5, 150.1], [14.4, 148.7], [14.5, 146.8], [13.8, 145.2], [14.5, 144.6], [14.4, 143.5],
                  [14.3, 141.6], [14.2, 140.6], [13.6, 139.2], [13.5, 138.0], [13.7, 137.2], [14.8, 136.6],
                  [15.5, 135.9], [16.1, 135.4], [16.8, 134.7], [17.7, 133.3], [18.5, 132.5], [19.7, 131.5], [20.8, 130.4]]) #출력

In [ ]:
#데이터 전처리
chaba_xdata = np.asarray(chaba_xdata, dtype = np.float32)
scaler = MinMaxScaler(feature_range=(0., 1.))
chaba_xdata = scaler.fit_transform(chaba_xdata)
chaba_xdata = np.reshape(chaba_xdata, (-1, 3, 1))

chaba_realdata = np.reshape(chaba_realdata, (-1, 2))

In [ ]:
yhat = model.predict(chaba_xdata, verbose = 0)

plt.plot(chaba_realdata, label = "real")
plt.plot(yhat,label="predcition")
plt.legend()
plt.show()